In [ ]:
!pip install orjson

In [27]:
!pip install PyMuPDF
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 6.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 7.3 MB/s eta 0:00:00:00:0100:01


In [7]:
!pip install jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 439.6 kB/s eta 0:00:00 0:00:01m


In [2]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
from functools import partial
import random
from typing import Dict, Tuple, Callable
from PIL import Image, UnidentifiedImageError
from typing import List, Optional
import pypdf
import orjson
import jsonlines     
import fitz  # PyMuPDF


In [15]:
json_path = Path('/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/train.jsonl')

# Open the JSONL file for reading
with jsonlines.open(json_path) as reader:
    for line_number, line in enumerate(reader, start=1):
        # Process each line as needed
        print(line['pdf'])

print(line_number)


/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/pdf/2201.00001.pdf
/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/pdf/2201.00012.pdf
/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/pdf/2201.00034.pdf
/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/pdf/2201.00070.pdf
4


In [ ]:
"""
Copyright (c) Meta Platforms, Inc. and affiliates.

This source code is licensed under the MIT license found in the
LICENSE file in the root directory of this source tree.
"""
# Implements image augmentation

import albumentations as alb
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD


def alb_wrapper(transform):
    def f(im):
        return transform(image=np.asarray(im))["image"]

    return f


class Erosion(alb.ImageOnlyTransform):
    """
    Apply erosion operation to an image.

    Erosion is a morphological operation that shrinks the white regions in a binary image.

    Args:
        scale (int or tuple/list of int): The scale or range for the size of the erosion kernel.
            If an integer is provided, a square kernel of that size will be used.
            If a tuple or list is provided, it should contain two integers representing the minimum
            and maximum sizes for the erosion kernel.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, scale, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        if type(scale) is tuple or type(scale) is list:
            assert len(scale) == 2
            self.scale = scale
        else:
            self.scale = (scale, scale)

    def apply(self, img, **params):
        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, tuple(np.random.randint(self.scale[0], self.scale[1], 2))
        )
        img = cv2.erode(img, kernel, iterations=1)
        return img


class Dilation(alb.ImageOnlyTransform):
    """
    Apply dilation operation to an image.

    Dilation is a morphological operation that expands the white regions in a binary image.

    Args:
        scale (int or tuple/list of int): The scale or range for the size of the dilation kernel.
            If an integer is provided, a square kernel of that size will be used.
            If a tuple or list is provided, it should contain two integers representing the minimum
            and maximum sizes for the dilation kernel.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, scale, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        if type(scale) is tuple or type(scale) is list:
            assert len(scale) == 2
            self.scale = scale
        else:
            self.scale = (scale, scale)

    def apply(self, img, **params):
        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, tuple(np.random.randint(self.scale[0], self.scale[1], 2))
        )
        img = cv2.dilate(img, kernel, iterations=1)
        return img


class Bitmap(alb.ImageOnlyTransform):
    """
    Apply a bitmap-style transformation to an image.

    This transformation replaces all pixel values below a certain threshold with a specified value.

    Args:
        value (int, optional): The value to replace pixels below the threshold with. Default is 0.
        lower (int, optional): The threshold value below which pixels will be replaced. Default is 200.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, value=0, lower=200, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        self.lower = lower
        self.value = value

    def apply(self, img, **params):
        img = img.copy()
        img[img < self.lower] = self.value
        return img


train_transform = alb_wrapper(
    alb.Compose(
        [
            Bitmap(p=0.05),
            alb.OneOf([Erosion((2, 3)), Dilation((2, 3))], p=0.02),
            alb.Affine(shear={"x": (0, 3), "y": (-3, 0)}, cval=(255, 255, 255), p=0.03),
            alb.ShiftScaleRotate(
                shift_limit_x=(0, 0.04),
                shift_limit_y=(0, 0.03),
                scale_limit=(-0.15, 0.03),
                rotate_limit=2,
                border_mode=0,
                interpolation=2,
                value=(255, 255, 255),
                p=0.03,
            ),
            alb.GridDistortion(
                distort_limit=0.05,
                border_mode=0,
                interpolation=2,
                value=(255, 255, 255),
                p=0.04,
            ),
            alb.Compose(
                [
                    alb.Affine(
                        translate_px=(0, 5), always_apply=True, cval=(255, 255, 255)
                    ),
                    alb.ElasticTransform(
                        p=1,
                        alpha=50,
                        sigma=120 * 0.1,
                        alpha_affine=120 * 0.01,
                        border_mode=0,
                        value=(255, 255, 255),
                    ),
                ],
                p=0.04,
            ),
            alb.RandomBrightnessContrast(0.1, 0.1, True, p=0.03),
            alb.ImageCompression(95, p=0.07),
            alb.GaussNoise(20, p=0.08),
            alb.GaussianBlur((3, 3), p=0.03),
            alb.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
            ToTensorV2(),
        ]
    )
)
test_transform = alb_wrapper(
    alb.Compose(
        [
            alb.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
            ToTensorV2(),
        ]
    )
)


In [47]:
# import albumentations as alb
# from albumentations.pytorch import ToTensorV2

# def alb_wrapper(transform):
#     def f(im):
#         return transform(image=np.asarray(im))["image"]

#     return f

# train_transform = alb_wrapper(
#         alb.Compose(
#         [
#             alb.Compose(
#                 [alb.ShiftScaleRotate(shift_limit=0, scale_limit=(-.15, 0), rotate_limit=1, border_mode=0, interpolation=3,
#                                     value=[255, 255, 255], p=1),
#                 alb.GridDistortion(distort_limit=0.1, border_mode=0, interpolation=3, value=[255, 255, 255], p=.5)], p=.15),
#             # alb.InvertImg(p=.15),
#             alb.RGBShift(r_shift_limit=15, g_shift_limit=15,
#                         b_shift_limit=15, p=0.3),
#             alb.GaussNoise(10, p=.2),
#             alb.RandomBrightnessContrast(.05, (-.2, 0), True, p=0.2),
#             alb.ImageCompression(95, p=.3),
#             alb.ToGray(always_apply=True),
#             alb.Normalize((0.7931, 0.7931, 0.7931), (0.1738, 0.1738, 0.1738)),
#             # alb.Sharpen()
#             ToTensorV2(),
            
#         ]
#     )
# )
# test_transform = alb.Compose(
#     [
#         alb.ToGray(always_apply=True),
#         alb.Normalize((0.7931, 0.7931, 0.7931), (0.1738, 0.1738, 0.1738)),
#         # alb.Sharpen()
#         ToTensorV2(),
#     ]
# )

In [109]:

import numpy as np
from PIL import Image
import cv2
from torchvision import transforms


from PIL import ImageOps
from timm.models.swin_transformer import SwinTransformer
from torchvision.transforms.functional import resize, rotate

class enc():

    def __init__(self):
        self.input_size = [4480,672]
        self.align_long_axis = True
        self.training = True

    def crop_margin(self, img: Image.Image) -> Image.Image:
        data = np.array(img.convert("L"))
        data = data.astype(np.uint8)
        max_val = data.max()
        min_val = data.min()
        if max_val == min_val:
            return img
        data = (data - min_val) / (max_val - min_val) * 255
        gray = 255 * (data < 200).astype(np.uint8)

        coords = cv2.findNonZero(gray)  # Find all non-zero points (text)
        a, b, w, h = cv2.boundingRect(coords)  # Find minimum spanning bounding box
        return img.crop((a, b, w + a, h + b))

    # def to_tensor(self, img: Image.Image):
    #     if self.training:
    #         return train_transform
    #     else:
    #         return test_transform
    def to_tensor(self, img: Image.Image):
        if self.training:
            # return train_transform(image = img)['image']
            return transforms.ToTensor()(img)
        else:
            return transforms.ToTensor()(img)
# """
#     def prepare_input(self, pdf_path:str, random_padding: bool = False):
#         self.input_tensor = None
#         if pdf_path is None:
#             return
#         img_list = []
#         # Convert PDF to images using PyMuPDF
#         input_size = [896,672]
#         doc = fitz.open(pdf_path)
#         for page_number in range(len(doc)):
#             page = doc[page_number]
#             image = page.get_pixmap()
#             # Convert Pixmap to PIL Image
#             img = Image.frombytes("RGB", [image.width, image.height], image.samples)

#             # crop margins
#             try:
#                 img = self.crop_margin(img.convert("RGB"))
#             except OSError:
#                 # might throw an error for broken files
#                 return
#             if img.height == 0 or img.width == 0:
#                 return
#             if self.align_long_axis and (
#                 (input_size[0] > input_size[1] and img.width > img.height)
#                 or (input_size[0] < input_size[1] and img.width < img.height)
#             ):
#                 img = rotate(img, angle=-90, expand=True)
#             img = resize(img, min(input_size))
#             img.thumbnail((input_size[1], input_size[0]))
#             delta_width = input_size[1] - img.width
#             delta_height = input_size[0] - img.height
#             if random_padding:
#                 pad_width = np.random.randint(low=0, high=delta_width + 1)
#                 pad_height = np.random.randint(low=0, high=delta_height + 1)
#             else:
#                 pad_width = delta_width // 2
#                 pad_height = delta_height // 2
#             padding = (
#                 pad_width,
#                 pad_height,
#                 delta_width - pad_width,
#                 delta_height - pad_height,
#             )
#             padded_img = ImageOps.expand(img, padding)
#             img_list.append(padded_img)

#             page_tensor = self.to_tensor(ImageOps.expand(img, padding))
#             page_tensor = self.to_tensor(img)
#             print("Page Tensor: ", page_tensor.shape)

#             print("page_tensor_shape: ", page_tensor.shape)
#             if self.input_tensor is None:
#                self.input_tensor = page_tensor
#             else:
#                 self.input_tensor = torch.cat([self.input_tensor, page_tensor], dim=2)


#         # Calculate the padding to be added
#         target_shape = (3, self.input_size[0], self.input_size[1])
#         original_shape = self.input_tensor.shape
#         padding = [0, target_shape[2] - original_shape[2]]

#         # Apply padding using torch.nn.functional.pad
#         padded_tensor = torch.nn.functional.pad(self.input_tensor, padding)
#         img = tensor_to_image(self.input_tensor, '/home/husainmalwat/PDF_2_LaTeX/img_test_horizontal.png')

#         return padded_tensor
#     """

    def prepare_input(self, pdf_path:str, random_padding: bool = False):
        self.input_tensor = None
        if pdf_path is None:
            return
        input_size= [896,672]
        # Convert PDF to images using PyMuPDF
        doc = fitz.open(pdf_path)
        i=0
        for page_number in range(len(doc)):
            if i==4:
                break
            page = doc[page_number]
            image = page.get_pixmap()
            # Convert Pixmap to PIL Image
            img = Image.frombytes("RGB", [image.width, image.height], image.samples)

            # crop margins
            try:
                img = self.crop_margin(img.convert("RGB"))
            except OSError:
                # might throw an error for broken files
                return
            if img.height == 0 or img.width == 0:
                return
            if self.align_long_axis and (
                (input_size[0] > input_size[1] and img.width > img.height)
                or (input_size[0] < input_size[1] and img.width < img.height)
            ):
                img = rotate(img, angle=-90, expand=True)
            img = resize(img, min(input_size))
            img.thumbnail((input_size[1], input_size[0]))
            delta_width = input_size[1] - img.width
            delta_height = input_size[0] - img.height
            if random_padding:
                pad_width = np.random.randint(low=0, high=delta_width + 1)
                pad_height = np.random.randint(low=0, high=delta_height + 1)
            else:
                pad_width = delta_width // 2
                pad_height = delta_height // 2
            padding = (
                pad_width,
                pad_height,
                delta_width - pad_width,
                delta_height - pad_height,
            )
            padded_img = ImageOps.expand(img, padding)

            # resize the image (224,224)
            # padded_img= padded_img.resize((224,224))
            page_tensor = self.to_tensor(padded_img)
            # page_tensor = self.to_tensor(ImageOps.expand(img, padding))
            print("Page Tensor: ", page_tensor.shape)
            if self.input_tensor is None:
               self.input_tensor = page_tensor
            else:
                self.input_tensor = torch.cat([self.input_tensor, page_tensor], dim=1)
            i+=1

        # VERTICAL PADDING added
        target_shape = (3, self.input_size[0], self.input_size[1])
        padding_needed = (target_shape[1] - self.input_tensor.size(1))
        self.input_tensor = torch.nn.functional.pad(self.input_tensor, (0, 0,0, padding_needed))


        img = tensor_to_image(self.input_tensor, '/home/husainmalwat/PDF_2_LaTeX/img_test_2.png')

        print("final Input Tensor: ", )
        return self.input_tensor


In [110]:
from torchvision.transforms import ToPILImage

def tensor_to_image(input_tensor, save_path):
    to_pil = ToPILImage()
    img = to_pil(input_tensor)
    img.save(save_path)

In [111]:
import json
class pdfDataset(Dataset):

    def __init__(self, dataset_path, split: str = "train"):
        super().__init__()
        self.dataset_path = dataset_path
        self.split = split
        self.pdf_path = []
        self.latex_path = []    

        with jsonlines.open(self.dataset_path) as reader:
            for line_number, line in enumerate(reader, start=1):
                self.pdf_path.append(line['pdf'])
                self.latex_path.append(line['latex'])
        self.dataset_length = line_number

    def __len__(self):
        return self.dataset_length

    def __getitem__(self, idx):

        pdf_path = self.pdf_path[idx]
        latex_path = self.latex_path[idx]

        # print("pdf: ", pdf_path)
        # print("latex: ", latex_path)
        encoder = enc()
        input_tensor = encoder.prepare_input(pdf_path, random_padding=True)

        # print("input_tensor: ", input_tensor)
        print("input_tensor.shape from getitem: ", input_tensor.shape)

        # with open(latex_path, "rb") as f:
        #     gnd_truth_data = f.read()
        #     try:
        #         gnd_truth_data = gnd_truth_data.decode("utf-8")  # Try decoding with UTF-8
        #     except:
        #         gnd_truth_data = gnd_truth_data.decode("latin-1", errors="ignore")  # Fallback to Latin-1, ignore errors

        # # print("gnd_truth_data: ", gnd_truth_data)
        # tokenizer_out = self.nougat_model.decoder.tokenizer(
        #     gnd_truth_data,
        #     max_length=self.max_length,
        #     padding="max_length",
        #     return_token_type_ids=False,
        #     truncation=True,
        #     return_tensors="pt",
        # )
        # input_ids = tokenizer_out["input_ids"].squeeze(0)
        # attention_mask = tokenizer_out["attention_mask"].squeeze(0)
        """      
        # randomly perturb ground truth tokens
        if self.split == "train" and self.perturb:
            # check if we perturb tokens
            unpadded_length = attention_mask.sum()
            while random.random() < 0.1:
                try:
                    pos = random.randint(1, unpadded_length - 2)
                    token = random.randint(
                        23, len(self.nougat_model.decoder.tokenizer) - 1
                    )
                    input_ids[pos] = token
                except ValueError:
                    break"""
        return input_tensor



        # Save input_tensor to JSON file
        # json_path = '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/Experiments/dataloader.json'
        # with open(json_path, "w") as file:
        #     json.dump(input_tensor.tolist(), file)
        


        return pdf_path, latex_path

In [112]:
dataset = pdfDataset(dataset_path="/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/train.jsonl")

In [113]:
for i, sample in enumerate(dataset):
    # print("iteration::",i, sample)
    break



Page Tensor:  torch.Size([3, 896, 672])
Page Tensor:  torch.Size([3, 896, 672])
Page Tensor:  torch.Size([3, 896, 672])
Page Tensor:  torch.Size([3, 896, 672])
final Input Tensor: 
input_tensor.shape from getitem:  torch.Size([3, 4480, 672])


In [114]:
## Padding Details
import json
json_path = '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/Experiments/dataloader.json'
with open(json_path, "r") as file:
    loaded_data = json.load(file)



for j in range(len(loaded_data[0])):
    count =0
    for i in range(len(loaded_data[0][0])):
        if loaded_data[0][j][i]!=0 and loaded_data[0][0][i]!=1:
            # print(loaded_data[0][0][i])
            count+=1
    print("count: ", count)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/Experiments/dataloader.json'

In [ ]:

"""class pdfDataset(Dataset):

    def __init__(self, 
                dataset_path,
                split: str = "traisn",
                root_name: str = "pdf",):
        super().__init__()
        self.dataset_path = dataset_path
        self.split = split
        self.root_name = root_name
        self.path_to_root = Path(dataset_path).parent
        self.root_name = root_name
        # self.path_to_root = self.path_to_root.parent

        # if not split in self.dataset_path:
        #     pti = self.path_to_root / (template % split + ".jsonl")
        #     if pti.exists():
        #         self.dataset_path = pti
        #     else:
        #         raise ValueError(f'Dataset file for split "{split}" not found: {pti}')
        
        self.dataset_file = None  # mulitprocessing
        
        if self.dataset_file is None:
            self.dataset_file = Path(self.dataset_path).open()
        line = self.dataset_file.readline()
        data: Dict = orjson.loads(line)

        pdf_path: Path = self.path_to_root / self.root_name / data.pop("pdf")
        latex_path: Path = self.path_to_root / self.root_name / data.pop("latex")

        self.dataset = {"pdf": pdf_path, "ground_truth": latex_path}
        self.dataset_length = len(self.dataset)


    def __len__(self):
        return self.dataset_length

    def __getitem__(self, idx: int):

        sample = self.dataset[idx]
        # print("idx:", idx)
        # print("sample:", sample)
        return sample
        # if sample is None:
        #     # if sample is broken choose another randomly
        #     return self[random.randint(0, self.dataset_length - 1)]
        # if sample is None or sample["image"] is None or prod(sample["image"].size) == 0:
        #     input_tensor = None
        # else:
        #     input_tensor = self.nougat_model.encoder.prepare_input(
        #         sample["image"], random_padding=self.split == "train"
                
        #     )

        # tokenizer_out = self.nougat_model.decoder.tokenizer(
        #     sample["ground_truth"],
        #     max_length=self.max_length,
        #     padding="max_length",
        #     return_token_type_ids=False,
        #     truncation=True,
        #     return_tensors="pt",
        # )
        # input_ids = tokenizer_out["input_ids"].squeeze(0)
        # attention_mask = tokenizer_out["attention_mask"].squeeze(0)
        # # randomly perturb ground truth tokens
        # if self.split == "train" and self.perturb:
        #     # check if we perturb tokens
        #     unpadded_length = attention_mask.sum()
        #     while random.random() < 0.1:
        #         try:
        #             pos = random.randint(1, unpadded_length - 2)
        #             token = random.randint(
        #                 23, len(self.nougat_model.decoder.tokenizer) - 1
        #             )
        #             input_ids[pos] = token
        #         except ValueError:
        #             break
        # return input_tensor, input_ids, attention_mask
    def __iter__(self):
        for i in range(self.dataset_length):
            yield self[i]"""



In [1]:
import fitz  # PyMuPDF
from PIL import Image



def pdf_to_imgs(pdf_path):  
    doc = fitz.open(pdf_path)
    images = []
    for page_number in range(len(doc)):
        page = doc[page_number]
        image = page.get_pixmap()
        # Convert Pixmap to PIL Image
        img = Image.frombytes("RGB", [image.width, image.height], image.samples)
        images.append(img)
    return images



def load_latex_code(self, pdf_path):
    # Implement this function to extract LaTeX code from the PDF
    # You may use a PDF parsing library or any other method
    # Return the extracted LaTeX code as a string
    latex_code = "Your LaTeX code here"
    return latex_code

def images_to_tensor(self, images):
    # Convert a list of images to a PyTorch tensor
    tensor_list = [self.transform(image) for image in images]
    return torch.stack(tensor_list, dim=0)

imgs = pdf_to_imgs("/mnt/NAS/patidarritesh/swin_transformer.pdf")




In [2]:
import numpy as np
import cv2

def crop_margin(img: Image.Image) -> Image.Image:
        data = np.array(img.convert("L"))
        data = data.astype(np.uint8)
        max_val = data.max()
        min_val = data.min()
        if max_val == min_val:
            return img
        data = (data - min_val) / (max_val - min_val) * 255
        gray = 255 * (data < 200).astype(np.uint8)

        coords = cv2.findNonZero(gray)  # Find all non-zero points (text)
        a, b, w, h = cv2.boundingRect(coords)  # Find minimum spanning bounding box
        return img.crop((a, b, w + a, h + b))

In [3]:
image = imgs[0]

image = crop_margin(image.convert("RGB"))

# if ((input_size[0] > input_size[1] and img.width > img.height)
#                 or (input_size[0] < input_size[1] and img.width < img.height)
#             ):
#                 img = rotate(img, angle=-90, expand=True)

In [4]:
imgs[0].size

(612, 792)

In [6]:
image.size


(528, 606)

In [7]:
print("Height: ",image.height)
print("Width: ", image.width)

Height:  606
Width:  528


In [8]:
image.save("output.png")



In [17]:
imgs[0].save("output_1.png")

In [9]:
from torchvision.transforms.functional import resize, rotate
image = resize(image, min([896,224]))


/home/patidarritesh/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
image.size

(224, 257)

In [11]:
x=606*224/528
x

257.09090909090907

In [ ]:
def to_tensor(self, img: Image.Image):
        transform_list = []
        
        # Add Resize transformation if needed
        if self.training:
            transform_list.append(transforms.Resize((896, 224)))

        # Add ToTensor transformation
        transform_list.append(transforms.ToTensor())

        composed_transform = transforms.Compose(transform_list)
        return composed_transform(img)
    
    def prepare_input(self, pdf_path:str, random_padding: bool = False):
        self.input_tensor = None
        if pdf_path is None:
            return
        input_size= [896,672]
        # Convert PDF to images using PyMuPDF
        doc = fitz.open(pdf_path)
        i=0
        for page_number in range(len(doc)):
            if i==5:
                break
            page = doc[page_number]
            image = page.get_pixmap()
            # Convert Pixmap to PIL Image
            img = Image.frombytes("RGB", [image.width, image.height], image.samples)
            print("raw img: ",img.size)
            # crop margins
            try:
                img = self.crop_margin(img.convert("RGB"))
                print("img after cropt margin: ",img.size)
                print("-----------------------------------------------")
            except OSError:
                # might throw an error for broken files
                return
            if img.height == 0 or img.width == 0:
                return
            if self.align_long_axis and (
                (input_size[0] > input_size[1] and img.width > img.height)
                or (input_size[0] < input_size[1] and img.width < img.height)
            ):
                img = rotate(img, angle=-90, expand=True)
            img = resize(img, min(self.input_size))
            img.thumbnail((input_size[1], input_size[0]))
            delta_width = input_size[1] - img.width
            delta_height = input_size[0] - img.height
            if random_padding:
                pad_width = np.random.randint(low=0, high=delta_width + 1)
                pad_height = np.random.randint(low=0, high=delta_height + 1)
            else:
                pad_width = delta_width // 2
                pad_height = delta_height // 2
            padding = (
                pad_width,
                pad_height,
                delta_width - pad_width,
                delta_height - pad_height,
            )
            
            page_tensor = self.to_tensor(ImageOps.expand(img, padding))
            print("Final transformed page_tensor for single page: ",page_tensor.shape)
            if self.input_tensor is None:
               self.input_tensor = page_tensor
            else:
                self.input_tensor = torch.cat([self.input_tensor, page_tensor], dim=2)
            i+=1

        target_shape = (3, self.input_size[0], self.input_size[1])
        original_shape = self.input_tensor.shape
        padding = [0, target_shape[2] - original_shape[2]]

        # Apply padding using torch.nn.functional.pad
        self.input_tensor = torch.nn.functional.pad(self.input_tensor, padding)
        print("input_tensor  after paddig: ",self.input_tensor.shape)
        return self.input_tensor

In [3]:
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import ImageOps
from timm.models.swin_transformer import SwinTransformer

/home/patidarritesh/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = SwinTransformer(
            # img_size=self.input_size,
            # depths=self.encoder_layer,
            # window_size=self.window_size,
            # patch_size=self.patch_size,
            # embed_dim=self.embed_dim,
            # num_heads=self.num_heads,
            # num_classes=0,
        )

/home/patidarritesh/miniconda3/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
swin_state_dict = timm.create_model(
"swin_base_patch4_window12_384", pretrained=True
).state_dict()
new_swin_state_dict =model.state_dict()

In [7]:
# print("new_swin_state_dict: ",new_swin_state_dict.keys())
lst = new_swin_state_dict.keys()

for i in lst:

    print((i))
    # print("Window Size: ", model.layers[0].blocks[0].attn.window_size[0])
    print(i)

patch_embed.proj.weight
patch_embed.proj.weight
patch_embed.proj.bias
patch_embed.proj.bias
patch_embed.norm.weight
patch_embed.norm.weight
patch_embed.norm.bias
patch_embed.norm.bias
layers.0.blocks.0.norm1.weight
layers.0.blocks.0.norm1.weight
layers.0.blocks.0.norm1.bias
layers.0.blocks.0.norm1.bias
layers.0.blocks.0.attn.relative_position_bias_table
layers.0.blocks.0.attn.relative_position_bias_table
layers.0.blocks.0.attn.relative_position_index
layers.0.blocks.0.attn.relative_position_index
layers.0.blocks.0.attn.qkv.weight
layers.0.blocks.0.attn.qkv.weight
layers.0.blocks.0.attn.qkv.bias
layers.0.blocks.0.attn.qkv.bias
layers.0.blocks.0.attn.proj.weight
layers.0.blocks.0.attn.proj.weight
layers.0.blocks.0.attn.proj.bias
layers.0.blocks.0.attn.proj.bias
layers.0.blocks.0.norm2.weight
layers.0.blocks.0.norm2.weight
layers.0.blocks.0.norm2.bias
layers.0.blocks.0.norm2.bias
layers.0.blocks.0.mlp.fc1.weight
layers.0.blocks.0.mlp.fc1.weight
layers.0.blocks.0.mlp.fc1.bias
layers.0.bloc

In [4]:
x=swin_state_dict["layers.0.blocks.0.attn.relative_position_bias_table"]

In [40]:
pos_bias = x.unsqueeze(0)[0]
pos_bias.shape

torch.Size([529, 4])

In [26]:
import math
window_size = 7

In [27]:
old_len = int(math.sqrt(len(pos_bias)))
new_len = int(2 * window_size - 1)

In [29]:
print("old_len: ",old_len)
print("new_len: ",new_len)

old_len:  23
new_len:  13


In [30]:
  pos_bias = pos_bias.reshape(1, old_len, old_len, -1).permute(
                        0, 3, 1, 2
                    )

In [32]:
pos_bias.shape

torch.Size([1, 4, 23, 23])

In [33]:
  pos_bias = F.interpolate(
                        pos_bias,
                        size=(new_len, new_len),
                        mode="bicubic",
                        align_corners=False,
                    )

In [35]:
pos_bias.shape

torch.Size([1, 4, 13, 13])

In [37]:
   new_swin_state_dict[x] = (
                        pos_bias.permute(0, 2, 3, 1)
                        .reshape(1, new_len**2, -1)
                        .squeeze(0)
                    )

In [39]:
 new_swin_state_dict[x].shape

torch.Size([169, 4])

In [5]:
from timm.models.swin_transformer_v2 import SwinTransformerV2

In [17]:
modelV2 = SwinTransformerV2(
            # img_size=self.input_size,
            # depths=self.encoder_layer,
            # window_size=self.window_size,
            # patch_size=self.patch_size,
            # embed_dim=self.embed_dim,
            # num_heads=self.num_heads,
            # num_classes=0,
        )

In [18]:
swin_state_dictV2 = timm.create_model(
"swinv2_base_window12to24_192to384", pretrained=True
).state_dict()
new_swin_state_dictV2 =modelV2.state_dict()

In [33]:
print(modelV2.layers[0].blocks[0])

SwinTransformerV2Block(
  (attn): WindowAttention(
    (cpb_mlp): Sequential(
      (0): Linear(in_features=2, out_features=512, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=512, out_features=3, bias=False)
    )
    (qkv): Linear(in_features=96, out_features=288, bias=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=96, out_features=96, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
    (softmax): Softmax(dim=-1)
  )
  (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  (drop_path1): Identity()
  (mlp): Mlp(
    (fc1): Linear(in_features=96, out_features=384, bias=True)
    (act): GELU(approximate='none')
    (drop1): Dropout(p=0.0, inplace=False)
    (norm): Identity()
    (fc2): Linear(in_features=384, out_features=96, bias=True)
    (drop2): Dropout(p=0.0, inplace=False)
  )
  (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  (drop_path2): Identity()
)


In [34]:
lst = new_swin_state_dictV2.keys()
j=0
for i in lst:
    # if i.endswith("relative_position_bias_table") : # and modelV2.layers[0].blocks[0].attn.window_size[0] != 12
        print(i)
        # print("Window Size: ", modelV2.layers[0].blocks[0].attn.window_size[0])
        j+=1
    # print(list(i.split('.')))

patch_embed.proj.weight
patch_embed.proj.bias
patch_embed.norm.weight
patch_embed.norm.bias
layers.0.blocks.0.attn.logit_scale
layers.0.blocks.0.attn.q_bias
layers.0.blocks.0.attn.v_bias
layers.0.blocks.0.attn.cpb_mlp.0.weight
layers.0.blocks.0.attn.cpb_mlp.0.bias
layers.0.blocks.0.attn.cpb_mlp.2.weight
layers.0.blocks.0.attn.qkv.weight
layers.0.blocks.0.attn.proj.weight
layers.0.blocks.0.attn.proj.bias
layers.0.blocks.0.norm1.weight
layers.0.blocks.0.norm1.bias
layers.0.blocks.0.mlp.fc1.weight
layers.0.blocks.0.mlp.fc1.bias
layers.0.blocks.0.mlp.fc2.weight
layers.0.blocks.0.mlp.fc2.bias
layers.0.blocks.0.norm2.weight
layers.0.blocks.0.norm2.bias
layers.0.blocks.1.attn.logit_scale
layers.0.blocks.1.attn.q_bias
layers.0.blocks.1.attn.v_bias
layers.0.blocks.1.attn.cpb_mlp.0.weight
layers.0.blocks.1.attn.cpb_mlp.0.bias
layers.0.blocks.1.attn.cpb_mlp.2.weight
layers.0.blocks.1.attn.qkv.weight
layers.0.blocks.1.attn.proj.weight
layers.0.blocks.1.attn.proj.bias
layers.0.blocks.1.norm1.weight

In [16]:
print("swin_state_dictV2:",swin_state_dictV2.keys())

swin_state_dictV2: odict_keys(['patch_embed.proj.weight', 'patch_embed.proj.bias', 'patch_embed.norm.weight', 'patch_embed.norm.bias', 'layers.0.blocks.0.attn.logit_scale', 'layers.0.blocks.0.attn.q_bias', 'layers.0.blocks.0.attn.v_bias', 'layers.0.blocks.0.attn.cpb_mlp.0.weight', 'layers.0.blocks.0.attn.cpb_mlp.0.bias', 'layers.0.blocks.0.attn.cpb_mlp.2.weight', 'layers.0.blocks.0.attn.qkv.weight', 'layers.0.blocks.0.attn.proj.weight', 'layers.0.blocks.0.attn.proj.bias', 'layers.0.blocks.0.norm1.weight', 'layers.0.blocks.0.norm1.bias', 'layers.0.blocks.0.mlp.fc1.weight', 'layers.0.blocks.0.mlp.fc1.bias', 'layers.0.blocks.0.mlp.fc2.weight', 'layers.0.blocks.0.mlp.fc2.bias', 'layers.0.blocks.0.norm2.weight', 'layers.0.blocks.0.norm2.bias', 'layers.0.blocks.1.attn.logit_scale', 'layers.0.blocks.1.attn.q_bias', 'layers.0.blocks.1.attn.v_bias', 'layers.0.blocks.1.attn.cpb_mlp.0.weight', 'layers.0.blocks.1.attn.cpb_mlp.0.bias', 'layers.0.blocks.1.attn.cpb_mlp.2.weight', 'layers.0.blocks.1.a

In [20]:
for x in new_swin_state_dictV2:
              
                if (
                    x.endswith("relative_position_bias_table")
                    and modelV2.layers[0].blocks[0].attn.window_size[0] != 12
                ):
                    print(x)
                    
                    pos_bias = swin_state_dictV2[x].unsqueeze(0)[0]
                    old_len = int(math.sqrt(len(pos_bias)))
                    new_len = int(2 * window_size - 1)
                    pos_bias = pos_bias.reshape(1, old_len, old_len, -1).permute(
                        0, 3, 1, 2
                    )
                    pos_bias = F.interpolate(
                        pos_bias,
                        size=(new_len, new_len),
                        mode="bicubic",
                        align_corners=False,
                    )
                    new_swin_state_dictV2[x] = (
                        pos_bias.permute(0, 2, 3, 1)
                        .reshape(1, new_len**2, -1)
                        .squeeze(0)
                    )
                else:
                    new_swin_state_dictV2[x] = swin_state_dict[x]
            # modelV2.load_state_dict(new_swin_state_dict)